Knowledge base System

문서를 이용한 벡터DB 구성 실습

In [2]:
!pip install docx2txt

# 문서 로딩

In [102]:
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax.docx')
docu = loader.load()
# docu

In [4]:
# 문서의 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter( # 큰 것부터 자름
    chunk_size = 1500, # large 모델은 chunk_size를 키우기
    chunk_overlap = 200, # 평균 10%
)
splitter

In [5]:
doc_list = loader.load_and_split(text_splitter=splitter)

In [18]:
len(doc_list)

183

# 임베딩

In [6]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 db 적재 - 크로마DB, Chroma, 인메모리 DB -> restart

In [1]:
#!pip install langchain_chroma

In [9]:
from langchain_chroma import Chroma
database = Chroma.from_documents( # 새로 생성
    documents= doc_list,
    embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma'
)

In [10]:
database = Chroma(collection_name='chroma-tax',
    persist_directory='./chroma', embedding_function=embedding)

# Retrieval

In [103]:
query = '연봉 5천만원인 거주자의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)
# retrieved_docs

# augmentation

In [12]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [13]:
prompt = f"""[Identity]
-당신은 최고의 한국 소득세 전문가입니다.
-[context] 내용만 참조해서 사용자의 질문에 답변해 주세요

[context]
{retrieved_docs}
Question:{query}
""" 

In [14]:
response = llm.invoke(prompt)

In [15]:
response.content

'연봉 5천만원인 거주자의 소득세를 계산하기 위해서는 여러 항목을 고려해야 합니다. 기본적으로 한국의 소득세는 누진세 구조를 가지고 있으며, 세율은 과세표준에 따라 달라집니다. \n\n2023년 기준으로 대략적인 소득세 계산 방식을 제시하겠습니다:\n\n1. **소득세 누진세율(2023년 기준):**\n   - 1,200만 원까지: 6%\n   - 1,200만 원 초과 4,600만 원까지: 15%\n   - 4,600만 원 초과 8,800만 원까지: 24%\n   - 그 이상: 35% (여기서 해당하지 않음)\n\n2. **연봉 5천만원의 과세표준은 모두 소득세의 대상입니다.**\n\n3. **세액 계산:**\n   - 1,200만 원까지: 1,200만 원 × 6% = 72만 원\n   - 1,200만 원 초과 4,600만 원까지: (5,000 - 1,200) 만원 = 3,800만원 × 15% = 570만 원\n   - 합계: 72만 원 + 570만 원 = 642만 원\n\n따라서, 연봉 5천만원인 거주자의 소득세는 대략 **642만원**입니다. \n\n단, 여기에 기본 공제, 인적 공제, 특별 공제 등 다양한 공제가 적용될 수 있으므로 최종적으로는 공제를 고려한 후의 세액이 다를 수 있습니다. 이는 대략적인 계산이므로, 개인의 여러 사항을 고려한 정확한 계산은 세무 전문가와 상담하시길 권장합니다.'

# 2. table 표로 변경

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax_table.docx')
docu = loader.load()
#docu

In [17]:
from langchain_chroma import Chroma
database = Chroma.from_documents( # 새로 생성
    documents= doc_list,
    embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma2'
)

In [18]:
query = '연봉 5천만원인 거주자의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)
retrieved_docs

[Document(id='1551a47e-c009-4fc0-b4ca-4e41d7730448', metadata={'source': './tax_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 ~ 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 ~ 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 ~ 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 ~ 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 ~ 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 ~ 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\

In [19]:
response.content

'연봉 5천만원인 거주자의 소득세를 계산하기 위해서는 여러 항목을 고려해야 합니다. 기본적으로 한국의 소득세는 누진세 구조를 가지고 있으며, 세율은 과세표준에 따라 달라집니다. \n\n2023년 기준으로 대략적인 소득세 계산 방식을 제시하겠습니다:\n\n1. **소득세 누진세율(2023년 기준):**\n   - 1,200만 원까지: 6%\n   - 1,200만 원 초과 4,600만 원까지: 15%\n   - 4,600만 원 초과 8,800만 원까지: 24%\n   - 그 이상: 35% (여기서 해당하지 않음)\n\n2. **연봉 5천만원의 과세표준은 모두 소득세의 대상입니다.**\n\n3. **세액 계산:**\n   - 1,200만 원까지: 1,200만 원 × 6% = 72만 원\n   - 1,200만 원 초과 4,600만 원까지: (5,000 - 1,200) 만원 = 3,800만원 × 15% = 570만 원\n   - 합계: 72만 원 + 570만 원 = 642만 원\n\n따라서, 연봉 5천만원인 거주자의 소득세는 대략 **642만원**입니다. \n\n단, 여기에 기본 공제, 인적 공제, 특별 공제 등 다양한 공제가 적용될 수 있으므로 최종적으로는 공제를 고려한 후의 세액이 다를 수 있습니다. 이는 대략적인 계산이므로, 개인의 여러 사항을 고려한 정확한 계산은 세무 전문가와 상담하시길 권장합니다.'

# 질의의 정규화

In [31]:
# 질의의 품질을 높이기 위한 정규화 과정 추가
def normalize_query(q: str) -> str:
    # 숫자: '5천만원' → '5,000만원'
    q = q.replace("오천만원","5,000만원").replace("5천만원","5,000만원")
    # 용어: 연봉 → (과세) 과세표준 후보어 추가
    q = q.replace("연봉", "과세표준")
    q = q.replace("직장인", "거주자")
    # 의도 신호: 계산 키워드 보강
    if "계산" not in q:
        q = q.rstrip("요?").rstrip("?") + " 계산 기준과 누진공제를 적용해 계산"
    return q

In [32]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
query_n = normalize_query(query)
query_n

'과세표준 5,000만원인 거주자의 소득세는 얼마인가 계산 기준과 누진공제를 적용해 계산'

In [ ]:
retrieved_docs = database.similarity_search(query_n, k=5) # 함수 실행
retrieved_docs

[Document(id='1551a47e-c009-4fc0-b4ca-4e41d7730448', metadata={'source': './tax_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 ~ 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 ~ 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 ~ 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 ~ 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 ~ 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 ~ 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\

# query 정규화, 고도화 작업이 필요한 이유

In [34]:
query, query_n

('연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 '과세표준 5,000만원인 거주자의 소득세는 얼마인가 계산 기준과 누진공제를 적용해 계산')

# Retrival와 RetrievalQA 체인

In [44]:
retriever = database.as_retriever(
    search_type='similarity',
    search_kwargs = {'k':5}
    ) # 기본 리트리버
retriever.invoke(query_n)

[Document(id='1551a47e-c009-4fc0-b4ca-4e41d7730448', metadata={'source': './tax_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 ~ 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 ~ 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 ~ 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 ~ 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 ~ 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 ~ 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\

In [40]:
from langchain import hub
prompt = hub.pull('rlm/rag-prompt') # 범용 래그 프롬프트

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [41]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [45]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs = {'prompt':prompt}
)

In [ ]:
response = qa_chain.invoke({"query":query_n}) # 소득세법 문서 추출 체인

In [3]:
#response

# Retrieval를 위한 키워드 사전(용어사전)

In [50]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ['사람을 나타내는 표현 --> 거주자']

prompt_dic = ChatPromptTemplate.from_template(
    f'''
    사용자의 질문을 보고, 아래의 사전을 참고해서 사용자의 질문을 변경하세요.
    만약 변경할 필요가 없다고 판단되면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴하세요.
    사전: {dictionary}
    
    질문: {{question}}
    ''' 
)

In [51]:
dictionary_chain = prompt_dic | llm | StrOutputParser()
tax_chain = {'query':dictionary_chain} | qa_chain

In [52]:
query

'연봉 5천만원인 직장인의 소득세는 얼마인가요?'

In [53]:
new_query = dictionary_chain.invoke({'question': query})
new_query

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [55]:
tax_chain.invoke({'question':query})

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 84만원에 1,400만원을 초과하는 금액의 15%를 더하여 계산됩니다. 즉, 5천만원의 소득에 대해 84만원 + (5,000,000 - 14,000,000) x 0.15를 계산하면 됩니다. 최종적으로 세액은 84만원 + 715만원 = 약 799만원이 됩니다.'}

# 새로운 쿼리로 실습하기 
### 제59조의2(자녀세액공제) 

In [95]:
# query = "자녀가 두 명인 경우 자녀세액공제 금액은 얼마인가요?"
query = "자녀가 두 명인 경우 받을 수 있는 세액공제 금액은 얼마인가요?"

In [ ]:
retriever = database.as_retriever( # 리트리버 객체 생성 후 실행
    search_type='similarity',
    search_kwargs = {'k':5}
    ) # 기본 리트리버
retriever.invoke(query)

[Document(id='2893cd3d-f7e8-4338-8add-77cec091d9bf', metadata={'source': './tax.docx'}, page_content='③ 제1항에 따른 공제를 “재해손실세액공제”라 한다.\n\n④ 재해손실세액공제를 적용받으려는 자는 대통령령으로 정하는 바에 따라 관할 세무서장에게 신청할 수 있다.\n\n⑤ 관할 세무서장이 제4항의 신청을 받았을 때에는 그 공제할 세액을 결정하여 신청인에게 알려야 한다.\n\n⑥ 제4항의 신청이 없는 경우에도 제1항을 적용한다.\n\n⑦ 집단적으로 재해가 발생한 경우에는 대통령령으로 정하는 바에 따라 관할 세무서장이 조사결정한 자산상실비율에 따라 제1항을 적용한다.\n\n⑧ 재해손실세액공제에 관하여 필요한 사항은 대통령령으로 정한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제59조(근로소득세액공제) ①근로소득이 있는 거주자에 대해서는 그 근로소득에 대한 종합소득산출세액에서 다음의 금액을 공제한다. <개정 2014. 1. 1., 2015. 5. 13.>\n\n\n\n② 제1항에도 불구하고 공제세액이 다음 각 호의 구분에 따른 금액을 초과하는 경우에 그 초과하는 금액은 없는 것으로 한다.<신설 2014. 1. 1., 2015. 5. 13., 2022. 12. 31.>\n\n1. 총급여액이 3천 300만원 이하인 경우: 74만원\n\n2. 총급여액이 3천 300만원 초과 7천만원 이하인 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]. 다만, 위 금액이 66만원보다 적은 경우에는 66만원으로 한다.\n\n3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\

In [97]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 🔑 Retrieval을 위한 용어 사전
dictionary = [
    "아이 --> 자녀",
    "아이 수 --> 자녀 수",
    "첫째 --> 1명",
    "둘째 --> 2명",
    "셋째 --> 3명",
    "세액공제 --> 자녀세액공제"
]

# 🔧 사전 기반 질의 변환 프롬프트
prompt_dic = ChatPromptTemplate.from_template(
    f'''
    사용자의 질문을 보고, 아래의 사전을 참고해서 사용자의 질문을 변경하세요.
    만약 변경할 필요가 없다고 판단되면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴하세요.
    사전: {dictionary}
    
    질문: {{question}}
    '''
)

In [98]:
dictionary_chain = prompt_dic | llm | StrOutputParser()

In [99]:
new_query = dictionary_chain.invoke({"question": query})
print(new_query)

자녀가 2명인 경우 받을 수 있는 자녀세액공제 금액은 얼마인가요?


In [100]:
query, new_query

('자녀가 두 명인 경우 받을 수 있는 세액공제 금액은 얼마인가요?', '자녀가 2명인 경우 받을 수 있는 자녀세액공제 금액은 얼마인가요?')

In [101]:
retriever = database.as_retriever(
    search_type='similarity',
    search_kwargs = {'k':5}
    ) # 기본 리트리버
retriever.invoke(new_query)

[Document(id='2893cd3d-f7e8-4338-8add-77cec091d9bf', metadata={'source': './tax.docx'}, page_content='③ 제1항에 따른 공제를 “재해손실세액공제”라 한다.\n\n④ 재해손실세액공제를 적용받으려는 자는 대통령령으로 정하는 바에 따라 관할 세무서장에게 신청할 수 있다.\n\n⑤ 관할 세무서장이 제4항의 신청을 받았을 때에는 그 공제할 세액을 결정하여 신청인에게 알려야 한다.\n\n⑥ 제4항의 신청이 없는 경우에도 제1항을 적용한다.\n\n⑦ 집단적으로 재해가 발생한 경우에는 대통령령으로 정하는 바에 따라 관할 세무서장이 조사결정한 자산상실비율에 따라 제1항을 적용한다.\n\n⑧ 재해손실세액공제에 관하여 필요한 사항은 대통령령으로 정한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제59조(근로소득세액공제) ①근로소득이 있는 거주자에 대해서는 그 근로소득에 대한 종합소득산출세액에서 다음의 금액을 공제한다. <개정 2014. 1. 1., 2015. 5. 13.>\n\n\n\n② 제1항에도 불구하고 공제세액이 다음 각 호의 구분에 따른 금액을 초과하는 경우에 그 초과하는 금액은 없는 것으로 한다.<신설 2014. 1. 1., 2015. 5. 13., 2022. 12. 31.>\n\n1. 총급여액이 3천 300만원 이하인 경우: 74만원\n\n2. 총급여액이 3천 300만원 초과 7천만원 이하인 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]. 다만, 위 금액이 66만원보다 적은 경우에는 66만원으로 한다.\n\n3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\

In [84]:
dictionary_chain = prompt_dic | llm | StrOutputParser()
tax_chain = {'query':dictionary_chain} | qa_chain

In [85]:
tax_chain.invoke({'question':query})

{'query': '자녀가 2명인 경우 받을 수 있는 세액공제는 얼마인가요?',
 'result': '자녀가 2명인 경우 받을 수 있는 세액공제는 연 55만원입니다. 3명 이상인 경우는 55만원에 2명을 초과하는 1명당 연 40만원을 추가로 받을 수 있습니다.'}